In [1]:
# ref : https://github.com/jadianes/spark-py-notebooks


In [2]:
# Launch Spark ipython notebook

"""
** RUN SPARK NOTEBOOK : 

$ source activate J_env ;  cd NOTEBOOK ; IPYTHON_OPTS="notebook" /Users/GGV/spark/./bin/pyspark


** RUN SPARK NOTEBOOK and processing CSV files with Spark SQL 

$ source activate J_env ;  cd NOTEBOOK ; IPYTHON_OPTS='notebook' /Users/GGV/spark/./bin/pyspark --packages com.databricks:spark-csv_2.11:1.2.0

"""

'\n** RUN SPARK NOTEBOOK : \n\n$ source activate J_env ;  cd NOTEBOOK ; IPYTHON_OPTS="notebook" /Users/GGV/spark/./bin/pyspark\n\n\n** RUN SPARK NOTEBOOK and processing CSV files with Spark SQL \n\n$ source activate J_env ;  cd NOTEBOOK ; IPYTHON_OPTS=\'notebook\' /Users/GGV/spark/./bin/pyspark --packages com.databricks:spark-csv_2.11:1.2.0\n\n'

In [3]:
sc

In [4]:
import datetime as dt   
import time
import csv
import requests
import pandas as pd, numpy as np

# 1.  Spark read csv, dataframe, SparkSQL 

In [5]:
# pandas dataframe  

pdf=pd.read_csv('/Users/GGV/Desktop/test.csv')
pdf.head()

PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S

In [6]:
# Spark dataframe 

df = sqlCtx.createDataFrame([(1, 4), (2, 5), (3, 6)], ["A", "B"])
df.show()

+---+---+
|  A|  B|
+---+---+
|  1|  4|
|  2|  5|
|  3|  6|
+---+---+



In [7]:
# Spark read .csv as dataframe 

df_ = sc.textFile("/Users/GGV/Desktop/test.csv")
df_ = df_.map(lambda x: (x, )).toDF()
df_.show()

+--------------------+
|                  _1|
+--------------------+
|PassengerId,Pclas...|
|892,3,"Kelly, Mr....|
|893,3,"Wilkes, Mr...|
|894,2,"Myles, Mr....|
|895,3,"Wirz, Mr. ...|
|896,3,"Hirvonen, ...|
|897,3,"Svensson, ...|
|898,3,"Connolly, ...|
|899,2,"Caldwell, ...|
|900,3,"Abrahim, M...|
|901,3,"Davies, Mr...|
|902,3,"Ilieff, Mr...|
|903,1,"Jones, Mr....|
|904,1,"Snyder, Mr...|
|905,2,"Howard, Mr...|
|906,1,"Chaffee, M...|
|907,2,"del Carlo,...|
|908,2,"Keane, Mr....|
|909,3,"Assaf, Mr....|
|910,3,"Ilmakangas...|
+--------------------+
only showing top 20 rows



In [8]:
# read CSV with PySpark SQL 

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)


df_test = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('/Users/GGV/Desktop/test.csv')

In [9]:
type(df_test)

pyspark.sql.dataframe.DataFrame

In [10]:
df_test.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [11]:
df_test.columns

['PassengerId',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [12]:
df_test.show()

+-----------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|        892|     3|    Kelly, Mr. James|  male|34.5|    0|    0|          330911| 7.8292|     |       Q|
|        893|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|          363272|    7.0|     |       S|
|        894|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0|          240276| 9.6875|     |       Q|
|        895|     3|    Wirz, Mr. Albert|  male|27.0|    0|    0|          315154| 8.6625|     |       S|
|        896|     3|Hirvonen, Mrs. Al...|female|22.0|    1|    1|         3101298|12.2875|     |       S|
|        897|     3|Svensson, Mr. Joh...|  male|14.0|    0|    0|            7538|  9.225|     |       S|
|        898|     3|Connolly, Miss. Kate|femal

In [13]:
# make pyspark dataframe as SQL 

df_test.registerTempTable("numeric")

In [14]:
# query 

sqlContext.sql("""SELECT * FROM numeric limit 2 """).show()

+-----------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|  Fare|Cabin|Embarked|
+-----------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|        892|     3|    Kelly, Mr. James|  male|34.5|    0|    0|330911|7.8292|     |       Q|
|        893|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|363272|   7.0|     |       S|
+-----------+------+--------------------+------+----+-----+-----+------+------+-----+--------+



# Spark basics functions 

In [15]:
# count words 

import os


def counts(text_file):
	word_counts = text_file \
	    .flatMap(lambda line: line.split()) \
	    .map(lambda word: (word, 1)) \
	    .reduceByKey(lambda a, b: a)
	    
	print word_counts.collect()    
    
   
text_file2 = sc.textFile("/Users/GGV/Desktop/test.csv")
counts(text_file2)

[(u'1289,1,"Frolicher-Stehli,', 1), (u'John",male,,1,0,367227,7.75,,Q', 1), (u'1103,3,"Finoli,', 1), (u'Gertrud', 1), (u'1244,2,"Dibden,', 1), (u'1212,3,"Andersson,', 1), (u'909,3,"Assaf,', 1), (u'(Helen', 1), (u'Joseph",male,,0,0,240261,10.7083,,Q', 1), (u'Johannes",male,20,0,0,SOTON/O2', 1), (u'Howard)",female,45,1,0,350026,14.1083,,S', 1), (u'33112,36.75,,S', 1), (u'James",male,34.5,0,0,330911,7.8292,,Q', 1), (u'Nils",male,29,0,0,347467,7.8542,,S', 1), (u'1220,2,"Clarke,', 1), (u'William",male,36,0,0,A/5', 1), (u'Robert",male,23,0,0,C.A.', 1), (u'12998,25.7417,,C', 1), (u'(Charles', 1), (u'Mrs.', 1), (u'17757,227.525,C62', 1), (u'1102,3,"Andersen,', 1), (u'Metcalfe",male,36,0,0,242963,13,,S', 1), (u'Herbert",male,18,0,0,347090,7.75,,S', 1), (u'Joshua', 1), (u'Thomas', 1), (u'1058,1,"Brandeis,', 1), (u'3101263,7.85,,S', 1), (u'Einar', 1), (u'Ferdinand",male,21,0,0,350410,7.8542,,S', 1), (u'B55,C', 1), (u'Louise",female,1,1,2,SC/Paris', 1), (u'1290,3,"Larsson-Rondberg,', 1), (u'908,2,

In [36]:
# filter 

df__ = sc.textFile("/Users/GGV/Desktop/test.csv")
df__.filter(lambda x: 'Miller' in x).collect()

[u'957,2,"Corey, Mrs. Percy C (Mary Phyllis Elizabeth Miller)",female,,0,0,F.C.C. 13534,21,,S',
 u'1144,1,"Clark, Mr. Walter Miller",male,27,1,0,13508,136.7792,C89,C',
 u'1164,1,"Clark, Mrs. Walter Miller (Virginia McDowell)",female,26,1,0,13508,136.7792,C89,C']

In [49]:
# map 
# only take 5 of the elements in list 

df__ = sc.textFile("/Users/GGV/Desktop/test.csv")
df__.map(lambda x: x.split("u")).take(5)

[[u'PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked'],
 [u'892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,,Q'],
 [u'893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47,1,0,363272,7,,S'],
 [u'894,2,"Myles, Mr. Thomas Francis",male,62,0,0,240276,9.6875,,Q'],
 [u'895,3,"Wirz, Mr. Albert",male,27,0,0,315154,8.6625,,S']]

In [68]:
# substract 

df__ = sc.textFile("/Users/GGV/Desktop/test.csv")
normal_raw_data = df__.filter(lambda x: 'Miller' in x) 
attack_raw_data= df__.subtract(normal_raw_data)

print (attack_raw_data.take(5))
print ("====")
print (attack_raw_data.filter(lambda x: 'Miller' in x).collect())

[u'1087,3,"Karlsson, Mr. Julius Konrad Eugen",male,33,0,0,347465,7.8542,,S', u'1056,2,"Peruschitz, Rev. Joseph Maria",male,41,0,0,237393,13,,S', u'1134,1,"Spedden, Mr. Frederic Oakley",male,45,1,1,16966,134.5,E34,C', u'1187,3,"Angheloff, Mr. Minko",male,26,0,0,349202,7.8958,,S', u'976,2,"Lamb, Mr. John Joseph",male,,0,0,240261,10.7083,,Q']
====
[]


In [83]:
#  Cartesian product between two RDDs by using  cartesian transformation



df__ = sc.textFile("/Users/GGV/Desktop/test.csv")
df__.map(lambda x : x.split(","))
protocols = df__.map(lambda x: x[1]).distinct()
protocols.collect()


#csv_data = raw_data.map(lambda x: x.split(","))
#protocols = csv_data.map(lambda x: x[1]).distinct()
#protocols.collect()

[u'a', u'1', u'3', u'5', u'7', u'9', u'0', u'8', u'2', u'4', u'6']

In [85]:
# https://github.com/jadianes/spark-py-notebooks/blob/master/nb5-rdd-aggregations/nb5-rdd-aggregations.ipynb